In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
current_dir = os.getcwd()
dataset_dir = os.path.join(current_dir, os.pardir, 'dataset')
print(dataset_dir)
for dirname, _, filenames in os.walk(dataset_dir):
  for filename in filenames:
    print(os.path.join(dirname, filename))

c:\Users\chuan\OneDrive\Documents\MovieRecommenderSystem\KNN\train\..\dataset
c:\Users\chuan\OneDrive\Documents\MovieRecommenderSystem\KNN\train\..\dataset\movies.csv
c:\Users\chuan\OneDrive\Documents\MovieRecommenderSystem\KNN\train\..\dataset\ratings.csv


In [2]:
movies_df = pd.read_csv(dataset_dir + '\movies.csv')
ratings_df = pd.read_csv(dataset_dir + '\\ratings.csv')

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df.shape

(62423, 3)

In [5]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [8]:
ratings_df.shape

(25000095, 4)

In [9]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [10]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [11]:
# df: 1 movie with 1 rating per row (movies_df + ratings_df)
# df1: df - [genres, timestamp]
df = movies_df.merge(ratings_df)
df1 = df.drop(['genres', 'timestamp'], axis=1)
df1.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),2,3.5
1,1,Toy Story (1995),3,4.0
2,1,Toy Story (1995),4,3.0
3,1,Toy Story (1995),5,4.0
4,1,Toy Story (1995),8,4.0


In [12]:
# df2: number of ratings per movie
df2 = df1.groupby('title')['rating'].count().reset_index().rename(columns={'rating': 'ratingcount'})
df2.head()

,title,ratingcount
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",179
2,#1 Cheerleader Camp (2010),9
3,#Captured (2017),2
4,#Female Pleasure (2018),3


In [13]:
# df3: 1 movie with 1 rating per row with addition of totalRatings column (df1 + df2)
df3 = df1.merge(df2,on='title',how='left')
df3.head()

,movieId,title,userId,rating,ratingcount
0,1,Toy Story (1995),2,3.5,57309
1,1,Toy Story (1995),3,4.0,57309
2,1,Toy Story (1995),4,3.0,57309
3,1,Toy Story (1995),5,4.0,57309
4,1,Toy Story (1995),8,4.0,57309


In [14]:
# Calculate quantiles of ratingcount
# 10%, 25%, 50%, 75%, 85%, 100% of the movies have ratingcount <= ?
print(f'10% of count= {df3.ratingcount.quantile(.1)}')
print(f'25% of count= {df3.ratingcount.quantile(.25)}')
print(f'50% of count= {df3.ratingcount.quantile(.5)}')
print(f'75% of count= {df3.ratingcount.quantile(.75)}')
print(f'85% of count= {df3.ratingcount.quantile(.85)}')
print(f'100% of count= {df3.ratingcount.quantile(1)}')

df3.describe()

10% of count= 825.0
25% of count= 2986.0
50% of count= 9152.0
75% of count= 20757.0
85% of count= 30910.0
100% of count= 81491.0


,movieId,userId,rating,ratingcount
count,2.500010e+07,2.500010e+07,2.500010e+07,2.500010e+07
mean,2.138798e+04,8.118928e+04,3.533854e+00,1.492536e+04
std,3.919886e+04,4.679172e+04,1.060744e+00,1.643934e+04
min,1.000000e+00,1.000000e+00,5.000000e-01,1.000000e+00
25%,1.196000e+03,4.051000e+04,3.000000e+00,2.986000e+03
50%,2.947000e+03,8.091400e+04,3.500000e+00,9.152000e+03
75%,8.623000e+03,1.215570e+05,4.000000e+00,2.075700e+04
max,2.091710e+05,1.625410e+05,5.000000e+00,8.149100e+04


In [15]:
# Movies with fewer rating (< 10000 in this case) provide less information about user preferences, sparse data lead to unreliable insights
# df4: subset of df3 with ratingcount >= 10000
df4 = df3[df3.ratingcount>=10000]
df4.shape

(11878413, 5)

In [16]:
df4.describe()

,movieId,userId,rating,ratingcount
count,1.187841e+07,1.187841e+07,1.187841e+07,1.187841e+07
mean,1.095668e+04,8.130033e+04,3.716982e+00,2.719677e+04
std,2.488045e+04,4.684746e+04,1.008191e+00,1.650535e+04
min,1.000000e+00,1.000000e+00,5.000000e-01,1.000500e+04
25%,5.930000e+02,4.071600e+04,3.000000e+00,1.434600e+04
50%,1.625000e+03,8.118400e+04,4.000000e+00,2.216500e+04
75%,4.878000e+03,1.217730e+05,4.500000e+00,3.457200e+04
max,1.641790e+05,1.625410e+05,5.000000e+00,8.149100e+04


In [17]:
# create pivot table, rows = movies, columns = users, values = ratings (if n/a, fill with 0)
movie_rating_pivote = df4.pivot_table(index='title',columns='userId', values='rating').fillna(0)
movie_rating_pivote.head()

userId,1,2,3,4,5,6,7,8,9,10,...,162532,162533,162534,162535,162536,162537,162538,162539,162540,162541
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,2.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,5.0,4.0,0.0,4.0,0.0,5.0,3.0,4.5,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
28 Days Later (2002),0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

movie_rating_df = csr_matrix(movie_rating_pivote.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute',n_neighbors=11)
model_knn.fit(movie_rating_df)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=11)

In [19]:
movie_rating_df.shape

(588, 162109)

In [20]:
distance,indices = model_knn.kneighbors(movie_rating_pivote.iloc[np.random.choice(movie_rating_df.shape[0]),:].values.reshape(1,-1),n_neighbors=11)

for i in range(0, len(distance.flatten())):
  if i == 0:
    print(f'Recommendation for a movie: {movie_rating_pivote.index[indices.flatten()[i]]}')
  else:
    print(f'{i}: {movie_rating_pivote.index[indices.flatten()[i]]} with a distance {distance.flatten()[i]}')

Recommendation for a movie: Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan (2006)
1: Superbad (2007) with a distance 0.5361843637728134
2: 40-Year-Old Virgin, The (2005) with a distance 0.5384954860901732
3: 300 (2007) with a distance 0.5836248859967192
4: Kill Bill: Vol. 1 (2003) with a distance 0.5874825081164771
5: Casino Royale (2006) with a distance 0.589500230437975
6: Sin City (2005) with a distance 0.5900283280141074
7: Little Miss Sunshine (2006) with a distance 0.5910952951447619
8: Hangover, The (2009) with a distance 0.5911821743494574
9: No Country for Old Men (2007) with a distance 0.5931203857835372
10: Departed, The (2006) with a distance 0.5938633036938488
